In [1]:
import numpy as np 

## Relocation of the Npole
In this section, we use:\
The rotations around the cartesian axis
$$R_x(\theta) = \begin{bmatrix} 1 & 0 & 0\\ 0 & \cos\left(\theta\right) & -\sin\left(\theta\right) \\ 0 & \sin\left(\theta\right)& cos\left(\theta \right)\end{bmatrix} \hspace{1cm}
R_y(\theta) = \begin{bmatrix} \cos\left(\theta\right) & 0 & \sin\left(\theta\right)\\ 0 & 1 & 0 \\ -\sin\left(\theta\right) & 0 & cos\left(\theta \right)\end{bmatrix} \hspace{1cm}
R_z(\theta) = \begin{bmatrix} \cos\left(\theta\right) & -\sin\left(\theta\right) & 0\\ \sin\left(\theta\right) & \cos\left(\theta\right) & 0 \\ 0 & 0 & 1\end{bmatrix}$$


Transformation from cartesian to spherical and viceversa
$$\begin{bmatrix} x \\ y \\ z \end{bmatrix} = r \begin{bmatrix} \sin(\theta) \cos(\varphi) \\ \sin(\theta) \sin(\varphi) \\ \cos(\theta) \end{bmatrix}$$


Define the positive rotation matrices around the cartesian axis. This means that the rotation is according to the right hand rule. BUT! When you apply it to the data your rotate the data anticlockwise so is like the coordinate axis would rotate clockwise! Note that they are defined for the cartesian coordinates

In [2]:
"""
import math
def Rx(math.radians(psi)):
    return np.matrix([[ 1, 0           , 0           ],
                       [ 0, m.cos(psi), -m.sin(psi)],
                       [ 0, m.sin(psi), m.cos(psi)]])
  
def Ry(math.radians(psi)):
    return np.matrix([[ m.cos(psi), 0, m.sin(psi)],
                       [ 0          , 1, 0          ],
                       [-m.sin(psi), 0, m.cos(psi)]])
  
def Rz(math.radians(psi)):
    return np.matrix([[ m.cos(psi), -m.sin(psi), 0 ],
                       [ m.sin(psi), m.cos(psi) , 0 ],
                       [ 0          , 0           , 1 ]])
"""

'\nimport math\ndef Rx(math.radians(psi)):\n    return np.matrix([[ 1, 0           , 0           ],\n                       [ 0, m.cos(psi), -m.sin(psi)],\n                       [ 0, m.sin(psi), m.cos(psi)]])\n  \ndef Ry(math.radians(psi)):\n    return np.matrix([[ m.cos(psi), 0, m.sin(psi)],\n                       [ 0          , 1, 0          ],\n                       [-m.sin(psi), 0, m.cos(psi)]])\n  \ndef Rz(math.radians(psi)):\n    return np.matrix([[ m.cos(psi), -m.sin(psi), 0 ],\n                       [ m.sin(psi), m.cos(psi) , 0 ],\n                       [ 0          , 0           , 1 ]])\n'

Define the transformation from spherical to cartiesian coordinates and viceversa

In [3]:
#Numerical error of 1e-5 when combining both functions for the longitude vector

def spherical_to_cartesian(latitude, longitude):
    from numpy import sin, cos, radians
    """
    Converts spherical coordinates (theta, phi) to Cartesian coordinates (x, y, z). Assume R constant
    
    Parameters:
        theta (float): Polar angle in radians (0 <= theta <= pi)
        phi (float): Azimuthal angle in radians (0 <= phi < 2*pi)
        
    Returns:
        tuple: Cartesian coordinates (x, y, z)
    """

    earthRadius = 6371*(10**3)

    theta = radians(90-latitude).ravel()
    phi = radians(longitude).ravel()
    
    x = (earthRadius * sin(theta) * cos(phi)).reshape(longitude.shape)
    y = (earthRadius * sin(theta) * sin(phi)).reshape(longitude.shape)
    z = (earthRadius * cos(theta)).reshape(longitude.shape)
    
    return x, y, z

def cartesian_to_spherical(x, y, z):  
    """
    Converts Cartesian coordinates (x, y, z) to spherical coordinates (r, theta, phi).
    
    Parameters:
        x (float): X-coordinate
        y (float): Y-coordinate
        z (float): Z-coordinate
        
    Returns:
        tuple: Spherical coordinates (theta, phi). Assume R constant
    """
    # Radial distance
    #r = math.sqrt(x**2 + y**2 + z**2)
    # Radial distance in the surface
    earthRadius = 6371*(10**3)
    
    # Polar angle (theta) from the top theta=0 in the pole 
    theta = np.degrees(np.acos(z/earthRadius))
    
    # Azimuthal angle (phi)
    phi = np.degrees(np.atan2(y, x))  # atan2 handles the correct quadrant for (y, x)
    
    return 90-theta, phi

Let's define the complete transformation for $R_x(90^0)$:
This transformation brings the NP to the equator (-y axis). Or, it brings the z axis to y and the y axis to -z.

In [4]:
def polar_rotation_rx(latitude,longitude,psi):

    from math import cos, sin, radians
    """
    Rotates a set of geographical coordinates (latitude and longitude) around the x-axis by a specified angle (psi).

    Parameters:
    - latitude: A numpy array representing the latitude values in degrees.
    - longitude: A numpy array representing the longitude values in degrees.
    - psi: The angle of rotation in radians. This angle specifies how much the coordinates should be rotated around the x-axis.

    Returns:
    - A tuple of two masked numpy arrays:
      - The first array contains the rotated latitude values.
      - The second array contains the rotated longitude values.
      Both arrays retain the mask from the input latitude and longitude arrays, ensuring that any masked values in the input are also masked in the output.

    Description:
    The function first constructs a rotation matrix `Rx` for rotating points around the x-axis by the angle `psi`. It then converts the input spherical coordinates (latitude and longitude) 
    into Cartesian coordinates. After applying the rotation matrix to these Cartesian coordinates, the function converts the rotated Cartesian coordinates back into spherical coordinates (latitude and longitude). 
    Finally, it returns the rotated coordinates as masked arrays, preserving any masks from the input arrays.
    """
    Rx = np.matrix([[ 1, 0         , 0          ],
                [ 0, cos(radians(psi)), -sin(radians(psi))],
                [ 0, sin(radians(psi)), cos(radians(psi))]])
    
    x, y, z = spherical_to_cartesian(latitude,longitude)
    rot_cartesian_matrix = Rx @ np.array([x.ravel(),y.ravel(),z.ravel()])
    lat_r, lon_r = cartesian_to_spherical(rot_cartesian_matrix[0,:].reshape(longitude.shape),rot_cartesian_matrix[1,:].reshape(longitude.shape),rot_cartesian_matrix[2,:].reshape(longitude.shape))
    #assign the mask
    return lat_r, lon_r